In [ ]:
import psycopg

In [ ]:
conn= psycopg.connect('''
    host= localhost
    dbname= Adventureworks
    user= postgres
    password= 1234
''')

In [ ]:
def getAddressById(address, connection):
    cur= connection.execute('''
            select * from person.address a where a.addressid = %s
        ''', (address,))
    if not cur.fetchone():
        print(f'The address with id {address} dont exists')
        return False
    return cur.fetchone()

In [ ]:
#getAddressById(0, conn)

In [ ]:
def set_order_header(connection, salesorderid, customerid, billtoaddressid, shiptoaddressid, shipmethodid, subtotal):
    try:
        cur= connection.execute('''
            select * from sales.customer c where c.customerid = %s 
        ''', (customerid,))
        if not cur.fetchone():
            print(f'Customer with id {customerid} does not exists.')
        else:
            if getAddressById(billtoaddressid, conn) != False:
                if getAddressById(shiptoaddressid, conn) != False:
                    cur= connection.execute('select * from purchasing.shipmethod sm where sm.shipmethodid = %s', (shipmethodid,))
                    if not cur.fetchone():
                        print(f'Ship method with id {shipmethodid} does not exists')
                    else:
                        cur= connection.execute('''
                            insert into sales.salesorderheader (salesorderid, duedate, customerid, billtoaddressid, shiptoaddressid, shipmethodid, subtotal)
                            values
                            (%s, NOW(), %s, %s, %s, %s, %s)
                        ''', (salesorderid, customerid, billtoaddressid, shiptoaddressid, shipmethodid, subtotal))
        return True
    except Exception as e:
        print(e)
        print('can\'t create order header')
        connection.rollback()
        return False

In [ ]:
set_order_header(conn, 1, 1, 1, 1, 1)

In [ ]:
def set_order_details(conn, num_items, orderheaderid, orderdetailid, productid, unitprice, orderqty, subtotal, specialofferid= 1):
    try:
        if unitprice <= 0 or orderqty <= 0:
            print('price and quantity must be a positive number')
        else:
            for i in range(num_items):
                cur= conn.execute('select * from sales.salesorderheader soh where soh.salesorderid = %s', (orderheaderid,))
                if not cur.fetchone():
                    print(f'The order header with id {orderheaderid} does not exists')
                else:
                    cur= conn.execute('select * from production.product p where p.productid = %s', (productid,))
                    if not cur.fetchone():
                        print(f'The product with id {productid} does not exists')
                    else:
                        cur= conn.execute('select * from sales.specialofferproduct sop where sop.specialofferid = %s and sop.productid = %s', (specialofferid,productid))
                        specialofferproduct= cur.fetchone()
                        if not specialofferproduct:
                            print(f'The product with id {productid} has no offer with id {specialofferid}')
                        else:
                            cur= conn.execute('select so.discountpct from sales.specialoffer so where so.specialofferid = %s', (specialofferproduct[0],))
                            unitpricediscount= unitprice*cur.fetchone()[0]
                            st= (unitprice-unitpricediscount)*orderqty
                            if st == subtotal:
                                cur= conn.execute('''
                                    insert into sales.salesorderdetail (salesorderid, salesorderdetailid, productid, specialofferid, unitprice, orderqty, unitpricediscount)
                                    values (%s, %s, %s, %s, %s, %s, %s)
                                ''', (orderheaderid, orderdetailid, productid, specialofferid, unitprice, orderqty, unitpricediscount))
                            else:
                                print('Subtotal don\'t match')
        return True
    except Exception as e:
        print(e)
        print('can\'t create order detail')
        conn.rollback()
        return False

In [ ]:
set_order_details(conn, num_items=1, orderheaderid=1, orderdetailid=1, productid=680, unitprice=1, orderqty=1, specialofferid=1)

In [ ]:
def make_transaction(connection):
    try:
        salesorderid= input('Enter order header id:')
        customerid= input('Enter customer id:')
        billtoaddressid= input('Enter bill to address id:')
        shiptoaddressid= input('Enter ship to address id:')
        shipmethodid= input('Enter ship method id:')
        subtotal= int(input('Enter the subtotal'))
        if set_order_header(connection= connection, salesorderid= salesorderid, customerid= customerid, billtoaddressid= billtoaddressid, shiptoaddressid= shiptoaddressid, shipmethodid= shipmethodid, subtotal= subtotal):
            num_items= int(input('Enter the numbers of order details to create'))
            orderdetailid= input('Enter the order detail id')
            productid= input('Enter the product id')
            unitprice= int(input('Enter the price of the product'))
            orderqty= int(input('Enter the numbers of products'))
            specialofferid= input('Enter the special offer id')
            if set_order_details(conn= connection, num_items= num_items, orderheaderid= salesorderid, orderdetailid= orderdetailid, productid= productid, unitprice= unitprice, orderqty= orderqty, specialofferid= specialofferid, subtotal= subtotal):
                connection.commit()
    except Exception as e:
        print(e)
        print('can\'t make the transaction')
        connection.rollback()

In [ ]:
make_transaction(connection= conn)

In [ ]:
conn.close()